# Week 1: DFS and BFS search, SCC

In [89]:
def get_data_from_text_file(path='data/graph_SCC.txt'):
    graph = {}
    graph_rev = {}
    with open(path) as f:
        for line in f:
            data = line.split()
            x = int(data[0])
            y = int(data[1])
            if x in graph:
                graph[x].append(y)
            else:
                graph[x] = [y]
            if y in graph_rev:
                graph_rev[y].append(x)
            else:
                graph_rev[y] = [x]
    return graph, graph_rev

In [90]:
def get_example_graphs_in_lecture():
    graph_rev = {}
    graph_rev[1] = [7]
    graph_rev[7] = [4, 9]
    graph_rev[4] = [1]
    graph_rev[9] = [6]
    graph_rev[6] = [3, 8]
    graph_rev[3] = [9]
    graph_rev[8] = [2]
    graph_rev[2] = [5]
    graph_rev[5] = [8]

    graph = {}
    graph[7] = [1]
    graph[4] = [7]
    graph[9] = [7, 3]
    graph[1] = [4]
    graph[6] = [9]
    graph[3] = [6]
    graph[8] = [6, 5]
    graph[2] = [8]
    graph[5] = [2]
    return graph, graph_rev

In [ ]:
# Practice: develop bfs using iterative and recursion methods

In [98]:
def dfs_recursive(g, u, visited, rank):
    # print(u)
    visited[u] = True
    if not u in g:
        rank.append(u)
        return
    for v in g[u]:
        if not v in visited:
            dfs_recursive(g, v, visited, rank)
    rank.append(u)
    
    
def dfs_iterative(g, u, visited, rank):
    stack = []
    stack.append(u)
    # print(u)
    visited[u] = True
    while len(stack) != 0:
        if not stack[-1] in g:
            v = stack.pop()
            rank.append(v)
        else:
            temp = False
            for w in g[stack[-1]]:
                if not w in visited:
                    stack.append(w)
                    # print(w)
                    visited[w] = True
                    temp = True
            if temp == False:
                v = stack.pop()
                rank.append(v)
    return visited, rank


def scc_finder(graph, graph_rev, dfs_method="iterative"):
    vortex_list = list(set(list(graph.keys()) + list(graph_rev.keys())))
    visited = {}
    rank_rev = []
    for node in vortex_list:
        if not node in visited:
            if dfs_method == "recursive":
                dfs_recursive(graph_rev, node, visited, rank_rev)
            elif dfs_method == "iterative":
                visited, rank_rev = dfs_iterative(graph_rev, node, visited, rank_rev)
    rank_rev = rank_rev[::-1]
    visited = {}
    rank_dir = []
    scc = []
    for node in rank_rev:
        if not node in visited:
            visited_prev_len = len(visited)
            if dfs_method == "recursive":
                dfs_recursive(graph, node, visited, rank_dir)
            elif dfs_method == "iterative":
                visited, rank_dir = dfs_iterative(graph, node, visited, rank_dir)
            scc.append(len(visited) - visited_prev_len)
    return sorted(scc, reverse=True)

In [92]:
# Getting data
run_mode = "homework"
if run_mode == "test":
    graph, graph_rev = get_example_graphs_in_lecture()
elif run_mode == "homework":
    graph, graph_rev = get_data_from_text_file()

In [99]:
scc = scc_finder(graph, graph_rev)
print(scc[:5])

[434821, 968, 459, 313, 211]


# Week 2: Shortest Path with Dijkstra

In [42]:
# This code uses the implementation of undirected graph from https://gist.github.com/econchick/4666413
# The algorithm itself is not derived from their solution
from collections import defaultdict

class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = defaultdict(list)
        self.distances = {}

    def add_node(self, value):
        self.nodes.add(value)

    def add_edge(self, from_node, to_node, distance):
        self.edges[from_node].append(to_node)
        # self.edges[to_node].append(from_node)
        self.distances[(from_node, to_node)] = distance


In [43]:
graph = Graph()
with open("dijkstra.txt") as f:
    for line in f:
        data = line.split()
        # print(data)
        from_node = int(data[0])
        graph.add_node(from_node)
        for to_node_str in data[1:]:
            to_node_lst = to_node_str.split(",")
            to_node = int(to_node_lst[0])
            distance = int(to_node_lst[1])
            graph.add_edge(from_node=from_node, 
                           to_node=to_node, 
                           distance=distance)            

In [44]:
# Initializing our algorithm
initial_node = 1

visited = set()
visited.add(initial_node)
A = {node: 1000000 for node in graph.nodes}
A[1] = 0

# 
not_finished = True
while not_finished:
    next_node = None
    minimum_distance = 1000000000
    for node_x in visited:
        for node_v in graph.edges[node_x]:
            if not node_v in visited:
                minimum_distance_test = A[node_x] + graph.distances[(node_x, node_v)]
                if minimum_distance_test < minimum_distance:
                    next_node = node_v
                    minimum_distance = minimum_distance_test
    if not next_node:
        not_finished = False
    else:
        A[next_node] = minimum_distance
        visited.add(next_node)

result = []
for node in [7,37,59,82,99,115,133,165,188,197]:
    result.append(A[node])
print(','.join([str(node) for node in result]))

2599,2610,2947,2052,2367,2399,2029,2442,2505,3068
